Install and Import Libraries

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')
print('Libraries imported.')

In [2]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Load list of London postcodes from Github

In [7]:
url_london="https://raw.githubusercontent.com/Issambkh/Capstone_Project/master/postcodes_london_GB.csv"
df_Geo_post_london = pd.read_csv(url_london,sep=";")
total_Br = df_Geo_post_london.shape[0] 
df_Geo_post_london.head(5)

,postcode,latitude,longitude,town,region,uk_region,country,country_string
0,BR1,51.41107,0.02192,Bromley,Greater London,London,ENG,England
1,BR2,51.38858,0.02237,Bromley,Greater London,London,ENG,England
2,BR3,51.40297,-0.03020,Bromley,Greater London,London,ENG,England
3,BR4,51.37559,-0.00695,Bromley,Greater London,London,ENG,England
4,BR5,51.38983,0.10436,Bromley,Greater London,London,ENG,England


In [3]:
Client_ID = 'VSXZSEDKHCRJVC031TBE3RU450NNUJCKPMYZDI0VDRFD0HAW'
Client_Secret = '1JDSLG0KLNLTXOINR2KDWC4VOZD21DJYWIYN4WVTPLPT1BNU'
VERSION_FS = '20181027' # Foursquare API version
radius = 2500
LIMIT = 2000

Explore London venues

In [7]:
latitude = df_Geo_post_london.iloc[0,1]
longitude = df_Geo_post_london.iloc[0,2]
url2 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(Client_ID,
                                                            Client_Secret, latitude, longitude , VERSION_FS, radius, LIMIT)
results = requests.get(url2).json()
items = results['response']['groups'][0]['items']
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered_all = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered_all['venue.categories'] = dataframe_filtered_all.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered_all.columns = [col.split('.')[-1] for col in dataframe_filtered_all.columns]
for x in range(1, total_Br):
    latitude = df_Geo_post_london.iloc[x,1]
    longitude = df_Geo_post_london.iloc[x,2]
    url2 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(Client_ID,
                                                            Client_Secret, latitude, longitude , VERSION_FS, radius, LIMIT)
    results = requests.get(url2).json()
    items = results['response']['groups'][0]['items']
    dataframe = json_normalize(items) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # filter the category for each row
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean columns
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
    dataframe_filtered_all = pd.concat([dataframe_filtered_all, dataframe_filtered],sort=False,ignore_index=True)
    City_x = df_Geo_post_london.iloc[x,4]
    Post_x = df_Geo_post_london.iloc[x,0]
    print(f'{x}: {City_x} with Postcode {Post_x} is explored')


1: Greater London with Postcode BR2 is explored
2: Greater London with Postcode BR3 is explored
3: Greater London with Postcode BR4 is explored
4: Greater London with Postcode BR5 is explored
5: Greater London with Postcode BR6 is explored
6: Greater London with Postcode BR7 is explored
7: Greater London with Postcode CR0 is explored
8: Greater London with Postcode CR2 is explored
9: Greater London with Postcode CR4 is explored
10: Greater London with Postcode CR5 is explored
11: Greater London with Postcode CR7 is explored
12: Greater London with Postcode CR8 is explored
13: Greater London with Postcode CR9 is explored
14: Greater London with Postcode DA14 is explored
15: Greater London with Postcode DA15 is explored
16: Greater London with Postcode DA17 is explored
17: Greater London with Postcode DA18 is explored
18: Greater London with Postcode DA5 is explored
19: Greater London with Postcode DA6 is explored
20: Greater London with Postcode DA7 is explored
21: Greater London with P

In [4]:
urlGreat_london_venue = "https://raw.githubusercontent.com/Issambkh/Capstone_Project/master/london_Restaurants_statistics.csv"
dataframe_filtered_all = pd.read_csv(urlGreat_london_venue)
dataframe_filtered_all = dataframe_filtered_all.drop('Unnamed: 0.1',axis=1)
Nr_venues = dataframe_filtered_all.shape[0]
dataframe_filtered_all.head(3)


,Unnamed: 0,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,...,lat,lng,neighborhood,postalCode,state,id,Checkin TipCount,venue price,venue price index,venue rating
0,0,The Red Lion,Pub,10 North Rd,GB,Bromley,United Kingdom,NaN,123,"['10 North Rd', 'Bromley', 'Greater London', '...",...,51.409972,0.021676,NaN,BR1 3LG,Greater London,4b634440f964a520996e2ae3,NaN,NaN,NaN,NaN
1,1,Barrel & Horn,Bar,204-206 High St,GB,Bromley,United Kingdom,NaN,808,"['204-206 High St', 'Bromley', 'Greater London...",...,51.405916,0.013724,NaN,BR1 1PW,Greater London,4ecd6c670aafd1868b867683,NaN,NaN,NaN,NaN
2,2,Cinnamon Culture,Indian Restaurant,46 Plaistow Lane,GB,Bromley,United Kingdom,NaN,355,"['46 Plaistow Lane', 'Bromley', 'Greater Londo...",...,51.414196,0.020883,NaN,BR1 3PA,Greater London,4d6ffb26b73bb1f79bacb272,7.0,Moderate,2.0,7.9


Get resaurants statistics

In [5]:
for x in range(1, Nr_venues):
    if 'Restaurant' in str(dataframe_filtered_all.iloc[x,2]):    
        ID_venue = dataframe_filtered_all.iloc[x,16]
        latitude = dataframe_filtered_all.iloc[x,11]
        longitude = dataframe_filtered_all.iloc[x,12]
        URL_VENUE = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&ll={},{}&v={}'.format(ID_venue, Client_ID, Client_Secret, latitude, longitude , VERSION_FS)
        results2 = requests.get(URL_VENUE).json()
        if 'venue' in results2['response']: 
            if 'stats' in results2['response']['venue']: 
                if 'tipCount' in results2['response']['venue']['stats']: 
                    dataframe_filtered_all.loc[x,'Checkin TipCount']= results2['response']['venue']['stats']['tipCount']
                if 'usersCount' in results2['response']['venue']['stats']: 
                    dataframe_filtered_all.loc[x,'Number Checkin']= results2['response']['venue']['stats']['usersCount']         
                if 'price' in results2['response']['venue']:   
                    dataframe_filtered_all.loc[x,'venue price']= results2['response']['venue']['price']['message']
                    dataframe_filtered_all.loc[x,'venue price index']= results2['response']['venue']['price']['tier']
                if 'rating' in results2['response']['venue']:
                    dataframe_filtered_all.loc[x,'venue rating']= results2['response']['venue']['rating']
        venuestat = dataframe_filtered_all.iloc[x,1]
        print('Venue %s Statistics collected' %venuestat)
        


Venue Hana Japanese Restaurant Statistics collected
Venue Crystal Restaurant Statistics collected
Venue La Saporita Statistics collected
Venue Ristorante Rossella Statistics collected
Venue Osteria Tufo Statistics collected
Venue Kipling's Statistics collected
Venue SSÄM - Korean Dining Statistics collected
Venue OA Comtam Statistics collected
Venue Xi'an Impression Statistics collected
Venue Le Péché Mignon Statistics collected
Venue Cookies and Scream Statistics collected
Venue Westerns Laundry Statistics collected
Venue Kokeb Ethiopian Cuisine Statistics collected
Venue Hana Japanese Restaurant Statistics collected
Venue Crystal Restaurant Statistics collected
Venue Prawn On The Lawn Islington Statistics collected
Venue CôBa Statistics collected
Venue Trullo Statistics collected
Venue Yida Sushi Statistics collected
Venue Black Axe Mangal Statistics collected
Venue Dotori Statistics collected
Venue Farang Statistics collected
Venue Osteria Tufo Statistics collected
Venue Pure Cyprus